In [ ]:
#%pip install sentence-transformers scikit-learn pandas

Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python39_64\python.exe -m pip install --upgrade pip' command.


In [2]:


import numpy as np
import pandas as pd
from sklearn.calibration import LabelEncoder
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score


print("Loading dataset...")

df = pd.read_csv("datasets/mood-data.csv") 
df = df[['text', 'label']] 


print("Dataset Loaded:")
print(df.head())

# ---------------------------------------------------------
# 2. Convert labels to integers (if needed)
# ---------------------------------------------------------

# If labels already 0–4, no need to encode, but let's be safe:
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

label_names = {0: "sadness", 1: "joy", 2: "love", 3: "anger", 4: "fear", 5:"surprise"}

df['label_name'] = df['label'].map(label_names)


X_train, X_test, y_train, y_test = train_test_split(
    df["text"], 
    df["label"], 
    test_size=0.2, 
    random_state=42,
    stratify=df["label"]
)

print("\nLoading SentenceTransformer model...")
embedder = SentenceTransformer('all-MiniLM-L6-v2')

print("Encoding text into embeddings...")
X_train_emb = embedder.encode(X_train.tolist(), batch_size=32, convert_to_numpy=True)
X_test_emb = embedder.encode(X_test.tolist(), batch_size=32, convert_to_numpy=True)


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_emb)
X_test_scaled = scaler.transform(X_test_emb)






Loading dataset...
Dataset Loaded:
                                                text  label
0                            i didnt feel humiliated      0
1  i can go from feeling so hopeless to so damned...      0
2   im grabbing a minute to post i feel greedy wrong      3
3  i am ever feeling nostalgic about the fireplac...      2
4                               i am feeling grouchy      3

Loading SentenceTransformer model...
Encoding text into embeddings...


In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Input(shape=(384,)),           # embedding size of MiniLM
    layers.Dense(256, activation="relu"),
    layers.Dropout(0.3),
    layers.Dense(128, activation="relu"),
    layers.Dropout(0.3),
    layers.Dense(6, activation="softmax")  # 6 emotion labels
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 256)            │        98,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 6)              │           774 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 132,230 (516.52 KB)

 Trainable params: 132,230 (516.52 KB)

 Non-trainable params: 0 (0.00 B)

In [4]:
history = model.fit(
    X_train_scaled,
    y_train,
    validation_split=0.1,
    epochs=20,
    batch_size=64,
    verbose=1
)


Epoch 1/20
180/180 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.5554 - loss: 1.2176 - val_accuracy: 0.6695 - val_loss: 0.9239
Epoch 2/20
180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6711 - loss: 0.9048 - val_accuracy: 0.6906 - val_loss: 0.8379
Epoch 3/20
180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7061 - loss: 0.7863 - val_accuracy: 0.6836 - val_loss: 0.8121
Epoch 4/20
180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7408 - loss: 0.7053 - val_accuracy: 0.7148 - val_loss: 0.7795
Epoch 5/20
180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7607 - loss: 0.6350 - val_accuracy: 0.7125 - val_loss: 0.7944
Epoch 6/20
180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7829 - loss: 0.5840 - val_accuracy: 0.7055 - val_loss: 0.7866
Epoch 7/20
180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8010 - loss: 0.5262 - val_accuracy: 0.7188 - val_loss: 0.7672
Epoch 8/20
180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8156 - loss: 0.4929 - val_accuracy: 0.

In [5]:
y_pred = model.predict(X_test_scaled)
y_pred_classes = np.argmax(y_pred, axis=1)

print("Accuracy:", accuracy_score(y_test, y_pred_classes))
print(classification_report(y_test, y_pred_classes))


100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Accuracy: 0.725625
              precision    recall  f1-score   support

           0       0.76      0.81      0.78       933
           1       0.73      0.83      0.78      1072
           2       0.59      0.41      0.48       261
           3       0.75      0.64      0.69       432
           4       0.69      0.64      0.66       387
           5       0.65      0.40      0.49       115

    accuracy                           0.73      3200
   macro avg       0.69      0.62      0.65      3200
weighted avg       0.72      0.73      0.72      3200

